#a)con COLAB


In [2]:

!pip install deap

import random
from deap import base, creator, tools, algorithms
import math

# Eliminar las clases anteriores si ya han sido creadas
try:
    del creator.FitnessMax
    del creator.Individual
except AttributeError:
    pass

# Definir la función de evaluación: f(x) = (x^(2x)) - 1
def eval_function(individual):
    x = individual[0]
    try:
        result = (x ** (2 * x)) - 1
    except OverflowError:
        result = float('inf')  # Evitar el desbordamiento en grandes números
    return result,

# Configuración de DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 1, 8)  # Limitar valores de x entre 1 y 8 para mayor variación
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registro de las operaciones genéticas
toolbox.register("mate", tools.cxTwoPoint)         # Cruce
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)  # Mutación con probabilidad del 10%
toolbox.register("select", tools.selTournament, tournsize=3)  # Selección por torneo
toolbox.register("evaluate", eval_function)


# cxTwoPoint modificado para manejar individuos con tamaño 1

def cxTwoPoint(ind1, ind2):

    size = min(len(ind1), len(ind2))
    if size > 1:  # Verifica si el individuo tiene más de un elemento para el cruce
        cxpoint1 = random.randint(1, size)
        cxpoint2 = random.randint(1, size - 1)
        if cxpoint2 >= cxpoint1:
            cxpoint2 += 1
    else:
        # Si el tamaño es 1, no se realiza el cruce
        cxpoint1, cxpoint2 = 0, 0

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2

# Reemplaza el cxTwoPoint existente en la caja de herramientas con esta versión modificada
toolbox.register("mate", cxTwoPoint)

# Algoritmo principal
def main():
    population = toolbox.population(n=10)  # Población inicial de 10 individuos
    num_generations = 3

    for generation in range(num_generations):
        print(f"Generación {generation + 1}")

        # Evaluar individuos
        for ind in population:
            ind.fitness.values = toolbox.evaluate(ind)
            print(f"Individuo {ind[0]}: f(x) = {ind.fitness.values[0]}")

        # Selección de la nueva generación
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Aplicar cruce y mutación
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:  # 50% probabilidad de cruce
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:  # 20% probabilidad de mutación
                toolbox.mutate(mutant)
                # Asegurar que el mutante siga dentro del rango válido
                mutant[0] = max(1, min(8, mutant[0]))  # Limitar a rango 1-8
                del mutant.fitness.values

        # Re-evaluar los descendientes
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)

        # Reemplazar la población con la nueva generación
        population[:] = offspring

    # Retornar la población final
    return population

# Ejecutar el algoritmo
final_population = main()

Generación 1
Individuo 1: f(x) = 0.0
Individuo 7: f(x) = 678223072848.0
Individuo 1: f(x) = 0.0
Individuo 3: f(x) = 728.0
Individuo 7: f(x) = 678223072848.0
Individuo 4: f(x) = 65535.0
Individuo 5: f(x) = 9765624.0
Individuo 8: f(x) = 281474976710655.0
Individuo 2: f(x) = 15.0
Individuo 1: f(x) = 0.0
Generación 2
Individuo 7: f(x) = 678223072848.0
Individuo 7: f(x) = 678223072848.0
Individuo 8: f(x) = 281474976710655.0
Individuo 5: f(x) = 9765624.0
Individuo 7: f(x) = 678223072848.0
Individuo 7: f(x) = 678223072848.0
Individuo 8: f(x) = 281474976710655.0
Individuo 4: f(x) = 65535.0
Individuo 2: f(x) = 15.0
Individuo 4: f(x) = 65535.0
Generación 3
Individuo 8: f(x) = 281474976710655.0
Individuo 7: f(x) = 678223072848.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Individuo 8: f(x) = 281474976710655.0
Indivi

#b)sin COLAB


In [3]:

import random
import math

# Función de evaluación: f(x) = (x^(2x)) - 1
def eval_function(x):
    return (x ** (2 * x)) - 1

# Crear la población inicial (números enteros aleatorios entre 1 y 10)
def create_population(size=10):
    return [random.randint(1, 10) for _ in range(size)]

# Selección por torneo
def tournament_selection(population, fitnesses, k=3):
    selected = []
    for _ in range(len(population)):
        # Elegir k individuos aleatorios y seleccionar el mejor
        competitors = random.sample(list(zip(population, fitnesses)), k)
        selected.append(max(competitors, key=lambda ind: ind[1])[0])
    return selected

# Cruce de dos puntos
def crossover(parent1, parent2):
    mask = (1 << random.randint(0, 4)) - 1
    child1 = (parent1 & ~mask) | (parent2 & mask)
    child2 = (parent2 & ~mask) | (parent1 & mask)
    return child1, child2

# Mutación (voltear un bit aleatorio)
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        bit_to_flip = 1 << random.randint(0, 4)
        individual ^= bit_to_flip  # XOR voltea el bit seleccionado
    return individual

# Algoritmo genético manual
def genetic_algorithm(num_generations=3, population_size=10):
    population = create_population(population_size)

    # Evolución a lo largo de generaciones
    for generation in range(num_generations):
        print(f"Generación {generation + 1}")

        # Evaluar fitness de cada individuo
        fitnesses = [eval_function(ind) for ind in population]
        for ind, fit in zip(population, fitnesses):
            print(f"Individuo {ind}: f(x) = {fit}")

        # Selección
        selected_population = tournament_selection(population, fitnesses)

        # Cruce (por pares)
        offspring = []
        for i in range(0, len(selected_population), 2):
            if i + 1 < len(selected_population):
                child1, child2 = crossover(selected_population[i], selected_population[i + 1])
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(selected_population[i])

        # Mutación
        population = [mutate(ind) for ind in offspring]

    # Retornar la población final
    return population

# Ejecutar el algoritmo manual
final_population_manual = genetic_algorithm()

Generación 1
Individuo 4: f(x) = 65535
Individuo 2: f(x) = 15
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 7: f(x) = 678223072848
Individuo 8: f(x) = 281474976710655
Individuo 10: f(x) = 99999999999999999999
Individuo 6: f(x) = 2176782335
Individuo 5: f(x) = 9765624
Individuo 8: f(x) = 281474976710655
Generación 2
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 10: f(x) = 99999999999999999999
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 10: f(x) = 99999999999999999999
Individuo 0: f(x) = 0
Generación 3
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 10: f(x) = 99999999999999999999
Individuo 8: f(x) = 281474976710655
Individuo 10: f(x) = 99999999999999999999
Individuo 8: f(x) = 281474976710655
Individuo 8: f(x) = 281474976710655
Individuo 10